# T-matrix computation for 3D acoustic scattering 

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import scipy.linalg.interpolative as sli

import cmath
import math
%matplotlib inline
import matplotlib.pyplot as plt
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
bempp.api.show_available_platforms_and_devices()
bempp.api.set_default_device(0, 0)

0: Portable Computing Language
    0: pthread-Intel(R) Xeon(R) W-2155 CPU @ 3.30GHz


In [2]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [3]:
d = [1,0,0] # wave's travel direction
k = 5 # wavenumber
eta = k
h = 0.1

In [4]:
# far field points: unit sphere
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = unit_sphere.vertices
space_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

# scatterer: cube
dom_cube = bempp.api.shapes.cube(length = 2/1.8, origin=(-1/1.8,-1/1.8,-1/1.8), h = h)
vert_cube = dom_cube.vertices
space_cube = bempp.api.function_space(dom_cube,'P',1)

identity = bempp.api.operators.boundary.sparse.identity(space_cube, space_cube,space_cube)
slp = bempp.api.operators.boundary.helmholtz.single_layer(space_cube,space_cube,space_cube,k)
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_cube,space_cube,space_cube,k)
slp_far = helmholtz_farfield.single_layer(space_cube, vert_sphere, k)
dlp_far = helmholtz_farfield.double_layer(space_cube, vert_sphere,k)

lhs = 0.5 * identity + dlp - 1j* eta *slp

mass_mat = bempp.api.operators.boundary.sparse.identity(space_sphere,space_sphere,space_sphere).weak_form().A

In [9]:
for deg in [10,11,12,13,14]:
    slp_far_field_coeff = []
    sph_harm_coeff = []
    
    T_exact = []
    T_appro = []
    mat1_exact = []
    mat1_appro = []

    for q in range(deg+1):
        for p in range(-q,q+1):
            @bempp.api.complex_callable
            def dirichlet_fun(x,n,domain_index,result):
                    with objmode():
                        result[0] = - regular_spherical_wavefunctions(p,q,x)

            rhs_fun = bempp.api.GridFunction(space_cube, fun = dirichlet_fun)
            field, info = bempp.api.linalg.gmres(lhs, rhs_fun, tol=1E-10)
            slp_far_field_coeff.append(dlp_far.evaluate(field) - 1j * eta * slp_far.evaluate(field))

            @bempp.api.complex_callable
            def sph_harm(x,n,domain_index,result):
                with objmode():
                    result[0] = normalized_spherical_harmonics(p,q,x)
            sph_harm_grid = bempp.api.GridFunction(space_sphere,fun = sph_harm)
            sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))
    T_matrix = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
    for i in range((deg+1)**2):
        for j in range((deg+1)**2):
            T_matrix[i,j] = (slp_far_field_coeff[j] @ mass_mat @ sph_harm_coeff[i])*k
    n = T_matrix.shape[0]
    nboxes = int(np.sqrt(n))
    size = n // nboxes
    
    
    P_exact,L_exact,U_exact = scipy.linalg.lu(T_matrix)
    logdet_exact = 0
    for i in range(T_matrix.shape[0]):
        logdet_exact += cmath.log(U_exact[i,i])
    logdet_exact1 = np.real(logdet_exact) + cmath.log(np.exp(np.imag(logdet_exact)*1j))
    
    mat1 = T_matrix + np.identity(T_matrix.shape[0])
    
    P_exact2,L_exact2,U_exact2 = scipy.linalg.lu(mat1)
    logdet_exact2 = 0
    for i in range(T_matrix.shape[0]):
        logdet_exact2 += cmath.log(U_exact2[i,i])

    logdet_exact22 = np.real(logdet_exact2) + cmath.log(np.exp(np.imag(logdet_exact2)*1j))
    
    T_exact.append(logdet_exact1)
    T_appro.append(logdet_(size,T_matrix,nboxes,n))
    mat1_exact.append(logdet_exact22)
    mat1_appro.append(logdet_(size,mat1,nboxes,n))
    print('============================')
    print(logdet_exact1)
    print(logdet_(size,T_matrix,nboxes,n))
    print(logdet_exact22)
    print(logdet_(size,mat1,nboxes,n))
    print('============================')

(-1053.5281526874721-2.5638044763652554j)
(-1053.528152687472-2.5638044763652843j)
(-10.418602047527472+2.3118739641249513j)
(-10.418602047527465+2.3118739641250134j)
(-1530.355589374248-1.006452524446436j)
(-1530.3555893742487+2.135140129143328j)
(-10.418602047182196+2.3118738371135916j)
(-10.418602047182194+2.3118738371136334j)
(-2142.1356580359543-2.615149665323774j)
(-2142.1356580359534+0.5264429882663868j)
(-10.418602047514373+2.311873828660542j)
(-10.418602047514348+2.3118738286606995j)
(-2908.4848078861414-1.1373190915799918j)
(-2908.484807886141-1.1373190915834142j)
(-10.418602047511953+2.3118738284954667j)
(-10.418602047511936+2.3118738284954343j)
(-3849.6694378152056+0.22451888763590896j)
(-3849.6694378152083-2.9170737659333215j)
(-10.41860204751543+2.3118738283759925j)
(-10.418602047515412+2.3118738283761435j)


In [5]:
def get_box(nx, ny, A, nboxes, n):
    """Return the content of a given box"""
    xrange = [nx * size, (1 + nx) * size] 
    yrange = [ny * size, (1 + ny) * size] 
    return A[xrange[0]:xrange[1], yrange[0]:yrange[1]] 

def get_con_box(i,A,nboxes,n):
    """Return Ai"""
    BB = A[i * size : (i + 1) * size, :]
    B = np.delete(BB, np.s_[i*size : (i+1)*size], axis = 1)
    
    CC = np.transpose(A[:, i * size : (i + 1) * size])
    C = np.delete(CC, np.s_[i*size : (i+1)*size], axis = 1)
    return np.hstack((B,C))

In [6]:
def L(i,k,A,nboxes,n):
    idx1,proj1 = sli.interp_decomp(get_con_box(i,A,nboxes,n), k, rand = False)
    B1 = sli.reconstruct_skel_matrix(get_con_box(i,A,nboxes,n),k,idx1)
    B1_trans = np.matrix.getH(B1)
    
    idx2,proj2 = sli.interp_decomp(B1_trans, k, rand = False)
    P2_trans = sli.reconstruct_interp_matrix(idx2,proj2)

    P2 = np.matrix.getH(P2_trans)
    return P2,idx2

def index_set(k,A,nboxes,n):
    l = []
    for i in range(nboxes):
        l.append(L(i,k,A,nboxes,n)[1][0:k])
    return l

def SSS(k,A,nboxes,n):
    ll = index_set(k,A,nboxes,n)
    mat = np.zeros((k*nboxes,k*nboxes),dtype = complex)
    for i in range(nboxes):
        for j in range(nboxes):
            if i != j:
                mat_ =  get_box(i,j,A,nboxes,n)[ll[i],:]
                mat[i*k:(i+1)*k,j*k:(j+1)*k] = mat_[:,ll[j]]
    return mat

def R(i,k,A,nboxes,n):
    RR = np.matrix.getH(L(i,k,A,nboxes,n)[0])
    return RR

def new_S(k,A,nboxes,n):
    mat_ = SSS(k,A,nboxes,n)
    for i in range(nboxes):
        mat_[i*k:(i+1)*k,i*k:(i+1)*k] = np.linalg.inv(R(i,k,A,nboxes,n) @ np.linalg.inv(get_box(i,i,A,nboxes,n)) @ L(i,k,A,nboxes,n)[0])
    return mat_

In [7]:
def logdet_(k,A,nboxes,n):
    a1 = 0
    p1,l1,u1 = scipy.linalg.lu(new_S(k,A,nboxes,n))
    for i in range(k*nboxes):
        a1 += cmath.log(u1[i,i])
        
    a2 = 0
    for i in range (nboxes):
        p2,l2,u2 = scipy.linalg.lu(get_box(i,i,A,nboxes,n))
        for j in range(size):
            a2 +=cmath.log(u2[j,j])
            
    a3 = 0
    for i in range(nboxes):
        p3,l3,u3 = scipy.linalg.lu(-R(i,k,A,nboxes,n)@np.linalg.inv(get_box(i,i,A,nboxes,n)) @L(i,k,A,nboxes,n)[0] )
        for j in range(k):
            a3 += cmath.log(u3[j,j])
    temp =  a1 + a2 + a3 - cmath.log((-1)**(k*nboxes))
    return np.real(temp) + cmath.log(np.exp(np.imag(temp)*1j))

In [10]:
def plot_box_ranks(T_matrix, nboxes, n, tol=1E-8):
    """Plot rank of the boxes."""
    from scipy.linalg import svdvals
    ranks = np.zeros((nboxes,nboxes),dtype='int32')
    for i in range(nboxes):
        for j in range(nboxes):
            m = get_box(i, j, T_matrix, nboxes, n)
            s = svdvals(m)
            ranks[i,j] = np.sum(s>tol)
    print(ranks)
    plt.imshow(ranks, cmap='Accent')
    plt.colorbar()